In [29]:
import json
import requests
import time
import urllib.request
import os

In [30]:
# initialize directories and log files
if not os.path.exists('processed_images'):
    os.makedirs('processed_images')

with open("log.txt", "w") as myfile:
    myfile.write("---------------------------------- BEGIN IMAGE PROCESSING JOB ----------------------------------\n")

In [31]:
# set image parameters
img1 = "https://img.lotlinx.com/vdn/7416/jeep_wrangler%20unlimited_2014_1C4BJWFG3EL326863_7416_339187295.jpg"
img2 = "https://img.lotlinx.com/vdn/7416/jeep_wrangler%20unlimited_2014_1C4BJWFG3EL326863_7416_2_339187295.jpg"
img3 = "https://img.lotlinx.com/vdn/7416/jeep_wrangler%20unlimited_2014_1C4BJWFG3EL326863_7416_3_339187295.jpg"
img4 = "https://img.lotlinx.com/vdn/7416/jeep_wrangler%20unlimited_2014_1C4BJWFG3EL326863_7416_4_339187295.jpg"
img5 = "https://img.lotlinx.com/vdn/7416/jeep_wrangler%20unlimited_2014_1C4BJWFG3EL326863_7416_5_339187295.jpg"
img_lst = [img1,img2,img3,img4,img5]
with open("log.txt", "a") as myfile:
        myfile.write("\n Reading Images from the following url's:\n {}\n".format(img_lst))

In [32]:
def json_builder(img_lst):
    dealerId = 1
    vid = 100
    dct = {}
    for url in img_lst:
        my_json_string = {'dealerId':dealerId,'vehicles':[{"id":vid,"images":[{"imageId":vid,"imageUrl":url}]}]}
        dct[dealerId] = my_json_string
        dealerId += 1
        vid = dealerId + 100
    with open("log.txt", "a") as myfile:
        myfile.write("\n JSON Body ready for Post Request:\n {}\n".format(dct))
    return dct

In [33]:
json_dct = json_builder(img_lst)

In [34]:
# send request
token_status = {}
for k,v in json_dct.items():
    json_body = v
    try:
        r = requests.post('https://photoai.lotlinx.com/images/optimize', json=json_body,auth=('testaccount5','984dc25e4e50'))
        rdct= r.json()
        token = rdct['data'][0]['token']
        status = rdct['data'][0]['status']
        token_status[token] = status

    except requests.exceptions.RequestException as e:
        print (e)
        with open("log.txt", "a") as myfile:
            myfile.write("\n Error in Post Request to optimize image:\n {}\n".format(e))

with open("log.txt", "a") as myfile:
    myfile.write("\n Image Optimization Post Request for the following is complete:\n {}\n".format(token_status))        

In [35]:
token_status

{'9DXH1V2wHXpS1tgwBMlyafkmenYlhzS9fp9KEEIyKvH7x7XHimlepFCIIs943Dol': 'queued',
 'ORFwetiyifZyTKX5EvWiJMP6wnfuTihcXoVMi9SJPBC49fxwGkbI1CL4lJ0XDlYH': 'queued',
 'WQoHPsLE5qkYe0pxPiErOADVvuXfQtkrly4ZTXXaBNinkCiz2gXSQxrykJKJkfoS': 'queued',
 'bp6F7gSvqsMODTEKp7avslY9BOnVuklWO4UruPhykZhb3PVnDljeLr5LNWQOBRAk': 'queued',
 'hcM1w8VV4fWy8qoPRmJA8VwHIA0mTQKlr5TTtLOl3p4XqvV5v8zvhskiXDQLNoAp': 'queued'}

In [36]:
def poll_api():
    count = 0
    stme = time.time()
    token_completed = []
    j=0
    print ("Status poll has started at ", stme)
    with open("log.txt", "a") as myfile:
        myfile.write("\n Status poll has started at {}\n".format(stme))

    
    
    while count <= 30: # poll for 1.5 min max
        for k,v in token_status.items():
            try:
                r = requests.get('https://photoai.lotlinx.com/images/{}/status'.format(k), auth=('testaccount5','984dc25e4e50'))
                rdct= r.json()
                token = rdct['data'][0]['token']
                status = rdct['data'][0]['status']
                with open("log.txt", "a") as myfile:
                    myfile.write("\n Status Poll Request:\n{}\n".format(rdct))
            except requests.exceptions.RequestException as e:
                print (e)
                with open("log.txt", "a") as myfile:
                    myfile.write("\n Error in Status Get Request:\n {}\n".format(e))

            
            if status == 'complete' and token not in token_completed: 
                modified_url = load_response(token)
                print(token,status,modified_url)
                with open("log.txt", "a") as myfile:
                    myfile.write("\n Completed Token :\n {}\n".format(token))
                    myfile.write("\n Completed New_url:\n{}\n".format(modified_url))
                token_completed.append(token)
                
                #save the new image
                try:
                    urllib.request.urlretrieve(modified_url,'processed_images/processed_image_{}'.format(j))
                    j+=1
                    with open("log.txt", "a") as myfile:
                        myfile.write("\n processed_image{} saved\n".format(j))
                except IOError as e:
                    myfile.write("\n Error while saving processed image :\n{}".format(e))
            
            elif status == 'failed':
                with open("log.txt", "a") as debug_log:
                    debug_log.write("\n Failed Token :\n {}\n".format(token))
           
                                          
            if len(token_completed) == len(token_status.keys()):
                stptme = time.time()
                print ("All images processed at  ", stptme)
                with open("log.txt", "a") as myfile:
                    myfile.write("\n All images processed at {}\n".format(stptme))
                    myfile.write("\n --------------------------------- END IMAGE PROCESSING JOB SUCCESS  ----------------------------------\n")
                return 1
                          
        count+=1
        time.sleep(3)
    stptme = time.time()
    print ("Status poll has run out of interations ", stptme)  
    with open("log.txt", "a") as myfile:
        myfile.write("\n Status poll has ended at {}\n".format(stptme))
        myfile.write("\n---------------------------------- END IMAGE PROCESSING JOB RAN OUT OF TIME ----------------------------------\n")


In [37]:
def load_response(token):
    r = requests.get('https://photoai.lotlinx.com/images/{}'.format(token), auth=('testaccount5','984dc25e4e50'))
    r = r.json()
    modified_url = r['data'][0]['vehicles'][0]['images'][0]['modifiedUrl']
    return modified_url

In [38]:
poll_api()

Status poll has started at  1526830143.2791777
hcM1w8VV4fWy8qoPRmJA8VwHIA0mTQKlr5TTtLOl3p4XqvV5v8zvhskiXDQLNoAp complete http://lotlinx-ml-images.s3.amazonaws.com/front_3_quarter/jeep_wrangler%2520unlimited_2014_1C4BJWFG3EL326863_7416_339187295.png
ORFwetiyifZyTKX5EvWiJMP6wnfuTihcXoVMi9SJPBC49fxwGkbI1CL4lJ0XDlYH complete http://lotlinx-ml-images.s3.amazonaws.com/side/jeep_wrangler%2520unlimited_2014_1C4BJWFG3EL326863_7416_3_339187295.png
WQoHPsLE5qkYe0pxPiErOADVvuXfQtkrly4ZTXXaBNinkCiz2gXSQxrykJKJkfoS complete http://lotlinx-ml-images.s3.amazonaws.com/front/jeep_wrangler%2520unlimited_2014_1C4BJWFG3EL326863_7416_2_339187295.png
9DXH1V2wHXpS1tgwBMlyafkmenYlhzS9fp9KEEIyKvH7x7XHimlepFCIIs943Dol complete http://lotlinx-ml-images.s3.amazonaws.com/side/jeep_wrangler%2520unlimited_2014_1C4BJWFG3EL326863_7416_5_339187295.png
bp6F7gSvqsMODTEKp7avslY9BOnVuklWO4UruPhykZhb3PVnDljeLr5LNWQOBRAk complete http://lotlinx-ml-images.s3.amazonaws.com/side/jeep_wrangler%2520unlimited_2014_1C4BJWFG3EL326863

1